In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
import sys
from xml.etree import ElementTree as et

In [3]:
def extract_from_list(list_of_dicts):
    final_dict = {}
    for dic in list_of_dicts:
        final_dict.update({dic['name']:dic['id']})
    return final_dict


#Scarica tutti gli indicatori della Banca Mondiale

url2 = 'http://api.worldbank.org/v2/indicators'
params2 = {
    'format':'json',
    'per_page':'400'
            }

start2 = 0
end2 = requests.get(url = url2, params = params2).json()[0]['pages']

indicators = {}
for page in range(start2, end2):
    if(page != 0):
        params2['page'] = page
    r2 = requests.get(url = url2, params = params2)
    if(r2.status_code == 200):
        list_of_dicts = r2.json()[1]
        indicators.update(extract_from_list(list_of_dicts))
        print(len(indicators), end = '\r')
    else:
        page = page-1
        time.sleep(1)

#Prendi solo gli indicatori riguardanti il GDP e la povertà

GDP_indicators = {}
poverty_indicators = {}
for key in indicators.keys():
    if('GDP' in str(key) or 'GNI' in key):
        GDP_indicators[key] = indicators[key]
    if('overty' in str(key)):
        poverty_indicators[key] = indicators[key]


In [4]:
def get_data_from_xml(indicator, start_date, end_date, iso_codes):  
    url3 = 'http://api.worldbank.org/countries/{}/indicators/{}'
    params3 = {'format' : 'xml', 'date' : '{}:{}'.format(start_date, end_date)}
    result = {}
    countries_noData = []
    
    for country in list(iso_codes.keys()):
        r3 = requests.get(url = url3.format(iso_codes[country],indicator), params = params3)
        result[country] = {}
        root = et.fromstring(r3.content)
        for child in root:
            result[country].update({child[2].text : child[3].text})
    return result

def nested_dict_to_dataframe(countries_dic):
    
    df = pd.DataFrame()
    
    for country in countries_dic.keys():
        items = countries_dic[country]
        if(type(items) is dict):
            values = list(items.values())
            cols = list(items.keys())
            df = df.append(pd.DataFrame(list(items.values()), index = list(items.keys()), columns = [country]).transpose())
            df = df.replace('No value', np.nan)
    return df

In [5]:
def insert_commas_as_thousands_sep(val):
    if(val is None):
        return None
    else:
        return "{:,}".format(float(val))


indicator_GDP = 'NY.GDP.MKTP.CD'
indicator_GNI = 'NY.GNP.MKTP.CD'
indicator_GNI_atlas = 'NY.GNP.ATLS.CD'
indicator_GNI_perCapita = 'NY.GNP.PCAP.KN'

#Recupero i dati dalle API su GDP, GNI, GNI calcolato col metodo Atlas e GNI pro capite 

# GDP_data = get_data_from_json(indicator_GDP, 2005, 2016, iso_codes)
# GNI_pc_data = get_data_from_json(indicator_GNI_perCapita, 2005, 2016, iso_codes)
# GNI_data = get_data_from_json(indicator_GNI, 2005, 2016, iso_codes)
# GNI_atlas_data = get_data_from_json(indicator_GNI_atlas, 2005, 2016, iso_codes)

GDP_data = get_data_from_xml(indicator_GDP, 2005, 2016, iso_codes)
GNI_data = get_data_from_xml(indicator_GNI, 2005, 2016, iso_codes)
GNI_atlas_data = get_data_from_xml(indicator_GNI_atlas, 2005, 2016, iso_codes)
GNI_pc_data = get_data_from_xml(indicator_GNI_perCapita, 2005, 2016, iso_codes)


GDP_frame = nested_dict_to_dataframe(GDP_data).dropna(how='all').applymap(insert_commas_as_thousands_sep)
GNI_frame = nested_dict_to_dataframe(GNI_data).dropna(how='all').applymap(insert_commas_as_thousands_sep)
GNI_atlas_frame = nested_dict_to_dataframe(GNI_atlas_data).dropna(how='all').applymap(insert_commas_as_thousands_sep)
GNI_pc_frame = nested_dict_to_dataframe(GNI_pc_data).dropna(how='all').applymap(insert_commas_as_thousands_sep)

folder = #Inserisci una cartella dove salvare i file sugli indicatori economici

GDP_frame.to_csv(folder.format('GDP'))
GNI_frame.to_csv(folder.format('GNI'))
GNI_atlas_frame .to_csv(folder.format('GNI_atlas'))
GNI_pc_frame.to_csv(folder.format('GNI_pc'))


ConnectionError: HTTPConnectionPool(host='api.worldbank.org', port=80): Max retries exceeded with url: /countries/TUR/indicators/NY.GDP.MKTP.CD?format=xml&date=2005%3A2016 (Caused by NewConnectionError("<urllib3.connection.HTTPConnection object at 0x0000027531404B00>: Failed to establish a new connection: [WinError 10060] Impossibile stabilire la connessione. Risposta non corretta della parte connessa dopo l'intervallo di tempo oppure mancata risposta dall'host collegato",))